<a href="https://colab.research.google.com/github/sharonma1218/stats-306/blob/main/lecture11_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(tidyverse)
library(rvest) # don't need to install this 

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




# Lecture 11: Web scraping

<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand how to import data from online sources by scraping web pages.
</div>

These notes correspond to Chapter 26 of your book.


## Ethics of scraping data online
You should carefully read [Section 26.2](https://r4ds.hadley.nz/webscraping.html#scraping-ethics-and-legalities) of the book concerning various ethical and legal issues surrounding scraping web sites for data. In this class we will only look at large, public web sites like Wikipedia and IMDB, where there is no risk of anything bad happening. However, there are other situations where it may be unethical, or even illegal, to harvest data from a website, even if you are technically able. **As data scientists in the real world, it will be up to you to carefully weigh these concerns before using the tools discussed in today's lecture.**

## Reading data from the Internet
These days, it's increasingly common to pull data from online sources. For example, say I wanted to know the population of European countries. This is [easily found](https://en.wikipedia.org/wiki/Demographics_of_Europe#Population_by_country) on Wikipedia. How can I get these data into R and analyze them?

## How do web pages work?

Web pages are written in a special language called HTML (**H**yper**t**ext **M**arkup **L**anguage). Here is a simple example of some HTML:

    <html>
    <head> 
      <title>Page title</title>
    </head>
    <body>
      <h1 id='first'>A heading</h1>
      <p>Some text &amp; <b>some bold text.</b></p>
      <img src='myimg.png' width='100' height='100'>
    </body>
    </html>

Web scraping is possible because most web pages have a consistent, hierarchical structure. For example, if I asked you how to navigate to the title of the web page shown above, you would follow the "path"

    html > head > title
    
to arrive at "Page title".

## HTML elements

There are a lot of HTML elements that might contain interesting information. Here are a few of the most common:
- Block tags that denote sections of text: `<h1>` (heading), `<p>` (paragraph), `<ul>`/`<ol>` (un)ordered list, etc.
- `<table>` (a table), `<tr>` (a table row), `<td>` (a table cell), etc.
- Each of these elements can contain attributes such as `id=` or `class=`. For example, `<table id="movies">` is probably a table that contains movie information.

The `rvest` package is used to load a web page and extract elements and tables based on their HTML tags. Let's see how it works by scraping the Wikipedia page mentioned earlier:

In [3]:
europop <- read_html("http://en.wikipedia.org/wiki/Demographics_of_Europe#Population_by_country") 
# the entire html pg

In this page there are many tables:

In [5]:
europop %>% html_elements("table")

{xml_nodeset (17)}
 [1] <table class="wikitable">\n<caption>Population of Europe, in millions, b ...
 [2] <table class="wikitable sortable" style="text-align:right;">\n<caption>P ...
 [3] <table class="wikitable sortable" style="text-align:right;">\n<caption>( ...
 [4] <table class="wikitable sortable" style="text-align: right;">\n<caption> ...
 [5] <table class="wikitable sortable static-row-numbers plainrowheaders srn- ...
 [6] <table class="nowraplinks mw-collapsible autocollapse navbox-inner" styl ...
 [7] <table class="nowraplinks mw-collapsible autocollapse navbox-inner" styl ...
 [8] <table class="nowraplinks mw-collapsible autocollapse navbox-inner" styl ...
 [9] <table class="nowraplinks hlist mw-collapsible autocollapse navbox-inner ...
[10] <table class="nowraplinks navbox-subgroup" style="border-spacing:0"><tbo ...
[11] <table class="nowraplinks navbox-subgroup" style="border-spacing:0"><tbo ...
[12] <table class="nowraplinks navbox-subgroup" style="border-spacing:0"><tbo .

How can we find the correct one? One option is to use our browser to find something that uniquely identifies the table that we want. Alternatively, since there are only about 17, we can just at each table until we find the one we want:

In [ ]:
wiki_tables<-europop%>%
  html_elements("table.wikitable.sortable")%>% # found thru inspect element 
  html_table
wiki_tables

Get third table, select the first two columns, slice off the first row using slice(), convert the datatypes of the columns using as.integer() & parse_number()



In [ ]:
wiki_tables[[3]]%>%
  select(1:3) %>% # however, the first row is repeated, so we want to remove it using slice
  slice(-1) %>% # every row except for the first one. but now we want to convert the <chr> cols to <num> so we use as.integer or parse_number
  mutate(Year=as.integer(Year),
Averagepopulation=parse_number(Averagepopulation),
b=parse_number('Live births'))

## 🤔 Quiz

What's the average population density ($\text{persons}/\text{km}^2$) for countries in Europe?

<ol style="list-style-type: upper-alpha;">
    <li>1234.5</li>
    <li>20000.0</li>
    <li>611.8</li>
    <li>6520.5</li>
    <li>101.1</li>
</ol>



In [19]:
wiki_tables[[4]] %>% colnames

[1] "Country (or territory)"                                    
[2] "Population[1][2]"                                          
[3] "Area.mw-parser-output .nobold{font-weight:normal}(km2)[14]"
[4] "Density(per km2)"                                          
[5] "Capital"

In [44]:
wiki_tables[[4]]%>% 
  select("Country (or territory)","Density(per km2)")%>%
  rename(country="Country (or territory)",
  density="Density(per km2)")%>%
  mutate(density=as.integer(density))

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `density = as.integer(density)`.
Caused by warning:
! NAs introduced by coercion”


avg_density
<dbl>
NA


In [43]:
wiki_tables[[4]]%>%
  select(dens='Density(per km2)')%>%
  mutate(dens=parse_number(dens))%>%
  summarize(avg=mean(dens))

avg
<dbl>
611.819


## 🤔 Quiz

Use the same page Wikipedia page (Demographics of Europe) to answer the following question:

On average, how many people were born *each day* in Europe between 2010 and 2021 (inclusive)?

<ol style="list-style-type: upper-alpha;">
    <li>90210.10</li>
    <li>23043.97</li>
    <li>7710127</li>
    <li>21123.64</li>
    <li>21109.18</li>
</ol>



In [48]:
library(lubridate)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [65]:
wiki_tables[[3]]%>%
  select(year='Year',births='Live births')%>%
  mutate(year=parse_number(year),
  births=parse_number(births))%>%
  filter(year>=2010 & year<=2021)%>%
  summarize(total=sum(births)) 

Warning message:
“There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `year = parse_number(year)`.
Caused by warning:
! 1 parsing failure.
row col expected actual
  1  -- a number   Year
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.”


total
<dbl>
92521523


In [68]:
difftime(make_date(2010,1,1),make_date(2022,1,1),"days") 

Time difference of -4383 days

In [67]:
92521523/4383

[1] 21109.18

## The Simpsons

The Simpsons is a popular and long-running TV show. How many people still watch the Simpsons? What is their most popular episode?

In [ ]:
simpsons <- read_html('https://en.wikipedia.org/wiki/List_of_The_Simpsons_episodes_(season_21–present)')

In [ ]:
# parse simpsons

## 🤔 Quiz

The episode with the largest number of viewers was **Once Upon a Time in Springfield**. Which episode of the Simpsons had the **smallest** number of viewers?


<ol style="list-style-type: upper-alpha;">
    <li>My Octopus and a Teacher</li>
    <li>Treehouse of Horror XXI</li>
    <li>Marge the Meanie</li>
    <li>The D'oh-cial Network</li>
    <li>The Devil Wears Nada</li>
</ol>



In [ ]:
# smallest number of viewers

## IMDB top movies

Let's consider a well-known table: the [top 250 movies on IMDB](https://www.imdb.com/chart/top/).

In [ ]:
imdb.250 <- read_html("https://www.imdb.com/chart/top/")

In [ ]:
# parse imdb

## Exercise

"The Kid" came out in 1921 and has a rating of 8.2. Another movie that was rated at least as high didn't come out until 1927 (Metropolis), so we could say that The Kid reigned as the #1 film for six years. Metropolis reigned for four years until City Lights (rating 8.4) came out.

Which film reigned for the longest amount of time?

In [ ]:
# longest reign

## Super Bowl TV ratings
We just had the Super Bowl. How have the TV ratings for the Super Bowl changed over the years?

In [ ]:
sbtv <- read_html('https://en.wikipedia.org/wiki/Super_Bowl_television_ratings') %>% html_elements('table') %>% .[[1]] %>% html_table

In [ ]:
# viewers over time

How does this compare with other major sports?

- https://en.wikipedia.org/wiki/World_Series_television_ratings
- https://en.wikipedia.org/wiki/NBA_Finals_television_ratings

In [ ]:
# super bowl vs world series

## Scraping other types of web data

Here are some examples of other types of web data we can scrape:

### The UofM Stats department
Let's say I wanted to make a table of all the [undergraduate stats courses](https://lsa.umich.edu/stats/undergraduate-students/statistics-courses.html) offered by the department. 

In [69]:
stats <- read_html('https://lsa.umich.edu/stats/undergraduate-students/statistics-courses.html')

How should we extract the data from this web page? We notice from inspecting the page that each course title is a `<b>` (bold) element:

In [74]:
# extract statistics courses
stats%>%
  html_elements("b")%>%
  html_text

[1] "STATS 100: Introduction to Statistics, Probability and Mathematical Modeling"
 [2] "STATS 125: Statistics, Games and Coincidences"                               
 [3] "STATS 150: Making Sense of Data"                                             
 [4] "STATS 180: AP Statistics"                                                    
 [5] "STATS 206/DATASCI 101: Introduction to Data Science"                         
 [6] "STATS 250: Introduction to Statistics and Data Analysis"                     
 [7] "STATS 251 – Introductory Statistics Supplement for Mathematics Education"    
 [8] "STATS 280: Honors Introduction to Statistics & Data Analysis"                
 [9] "STATS 299 - Workplace Internship for Undergraduate Statistics Majors"        
[10] "STATS/DATASCI 306: Introduction to Statistical Computing"                    
[11] "STATS/DATASCI 315 – Statistics and Artificial Intelligence "                 
[12] ""                                                                            
[13] "STATS 401: Applied Statistical Methods II"                                   
[14] "STATS 403: Introduction to Quantitative Research Methods"                    
[15] "STATS 404: Effective Communication in Statistics"                            
[16] "STATS/DATASCI 406: Computational Methods in Statistics and Data Science"     
[17] "STATS 412: Introduction to Probability & Statistics"                         
[18] "STATS/DATASCI 413: Applied Regression Analysis"                              
[19] "STATS 414: Topics Course"                                                    
[20] "STATS/DATASCI 415: Data Mining and Statistical Learning"                     
[21] "STATS/MATH 425: Introduction to Probability"                                 
[22] "STATS 426: Introduction to Theoretical Statistics"                           
[23] "STATS 430: Applied Probability"                                              
[24] "STATS/BIOSTAT 449: Topics in Biostatistics"                                  
[25] "STATS/DATASCI 451: Bayesian Data Analysis"                                   
[26] "STATS 470: Introduction to the Design of Experiments"                        
[27] "STATS 480: Survey Sampling Techniques"                                       
[28] "STATS/DATASCI 485: Capstone Seminar"                                         
[29] "STATS 489: Independent Study in Statistics"                                  
[30] "STATS 499: Honors Seminar"                                                   
[31] "Please send permission requests to statsugradprogram@umich.edu"

### Reddit
Let's see how to scrape the [UofM Reddit site](https://old.reddit.com/r/uofm):

In [ ]:
top.reddit <- read_html('https://old.reddit.com/r/uofm/top/?sort=top&t=all')

Let's plot the top scoring posts, when they were posted, and how many votes they have received.

In [ ]:
# top posts on r/uofm